In [16]:
let parseLine (line : string) = 
    line.Split(" -> ")
    |> Array.map (
        fun coords -> 
            let xy = coords.Split(',', 2)
            int xy[0], int xy[1]
            )

let steps a b =
    if a < b then
        [ a .. b ]
    else 
        [ b .. a ]

let connect (x1,y1) (x2,y2) =
    if x1 = x2 then
        steps y1 y2 |> Seq.map (fun y -> x1,y)
    else
        steps x1 x2 |> Seq.map (fun x -> x,y1)

let linePoints coords =
    coords
    |> Array.pairwise
    |> Seq.collect (fun (a, b) -> connect a b)            
    |> Set.ofSeq

type Cell = | Rock | Sand    

let parseBoard lines =
    lines
    |> Seq.map parseLine
    |> Seq.map linePoints
    |> Set.unionMany
    |> Seq.map (fun pos -> pos, Rock)
    |> Map.ofSeq

let rec trackFall outOfBounds (x,y) (board : Map<int*int, Cell>) =
    if y = outOfBounds then
        None
    else
        match board |> Map.tryFind (x, y+1) with
        | None ->
            trackFall outOfBounds (x, y+1) board
        | Some _ ->
            match board |> Map.tryFind (x-1, y+1) with
            | None -> trackFall outOfBounds (x-1, y+1) board
            | _ ->
                match board |> Map.tryFind (x+1, y+1) with
                | None -> trackFall outOfBounds (x+1, y+1) board
                | _ -> Some ((), board |> Map.add (x,y) Sand)

let countSand board = 
    let outOfBounds = 1 + (board |> Map.keys |> Seq.map snd |> Seq.max)

    board
    |> Seq.unfold (fun b -> trackFall outOfBounds (500,0) b)
    |> Seq.length



In [15]:
parseLine "503,4 -> 502,4 -> 502,9 -> 494,9" |> linePoints


index,Item1,Item2
0,502,4
1,503,4
2,502,4
3,502,5
4,502,6
5,502,7
6,502,8
7,502,9
8,494,9
9,495,9
